# Shelter Animal Outcomes 3

## Random Forests

In [1]:
from time import time
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.feature_selection import RFECV
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('../Shelter_train.csv')
df_test = pd.read_csv('../Shelter_test.csv')

In [3]:
X = df_train.ix[:, :-1]
y = df_train.ix[:, -1]
df_test = df_test.drop('ID', 1)

In [4]:
forest = RandomForestClassifier(n_estimators = 400, max_features='auto')
cross_validation.cross_val_score(forest, X, y, scoring="log_loss")

array([-1.46168807, -1.4526439 , -1.37033744])

In [5]:
params = {"clf__max_depth": [5, 3, None],
        "clf__max_features": [0.1, 0.25, 0.5, 1.0],
        "clf__min_samples_split": [1, 3, 10],
        "clf__min_samples_leaf": [1, 3, 10],
        "clf__bootstrap": [True, False],
        "clf__criterion": ["gini", "entropy"]}

In [6]:
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [7]:
pipeline = Pipeline([
        ('featureSelection', RFECV(estimator=RandomForestClassifier(n_estimators=20), scoring='log_loss')),
        ('clf', RandomForestClassifier(n_estimators=1000))
        ])
grid_search = GridSearchCV(pipeline, params, n_jobs=-1)
start = time()
grid_search.fit(X, y)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.grid_scores_)))
report(grid_search.grid_scores_)
predictions = grid_search.predict_proba(df_test)
output = pd.DataFrame(predictions, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output.index.names = ['ID']
output.index += 1
output.head()

GridSearchCV took 5479.77 seconds for 432 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.608 (std: 0.003)
Parameters: {'clf__bootstrap': True, 'clf__criterion': 'entropy', 'clf__max_depth': 5, 'clf__min_samples_leaf': 3, 'clf__max_features': 0.25, 'clf__min_samples_split': 10}

Model with rank: 2
Mean validation score: 0.608 (std: 0.003)
Parameters: {'clf__bootstrap': True, 'clf__criterion': 'gini', 'clf__max_depth': 3, 'clf__min_samples_leaf': 10, 'clf__max_features': 1.0, 'clf__min_samples_split': 10}

Model with rank: 3
Mean validation score: 0.608 (std: 0.003)
Parameters: {'clf__bootstrap': True, 'clf__criterion': 'entropy', 'clf__max_depth': 5, 'clf__min_samples_leaf': 1, 'clf__max_features': 1.0, 'clf__min_samples_split': 1}



,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.140022,0.013152,0.082529,0.184631,0.579666
2,0.574449,0.002360,0.025939,0.212823,0.184429
3,0.505899,0.001270,0.036873,0.288141,0.167818
4,0.108558,0.020495,0.104876,0.281734,0.484337
5,0.530612,0.001736,0.030262,0.245144,0.192246


In [8]:
output.to_csv('../submissionRF.3.0.csv', index_label = 'ID')